# Milestone 2 - Training Transformer Models with GloVe embeddings

### Imports

In [ ]:
%cd ..
# imports
from scripts_m2 import *


### 1. Get Dataloaders

In [ ]:
train_dataloader, dev_dataloader, tokenizer = prepare_dataloaders_tokenizer_glove()

### 2. Model Training

#### Question [PAD] [SEP] Context [PAD]

In [ ]:
print(tokenizer.get_pretrained_embeddings().shape)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model = TransformerQAModel3(
    vocab_size=10_000,
    d_model=300,
    num_layers=4,
    num_heads=6,
    dim_feedforward=512,
    max_question_len=23,
    max_context_len=318,
    dropout=0.01,
    pretrained_embeddings= tokenizer.get_pretrained_embeddings(),
    freeze_embeddings=False,
)
model = model.to(device)

# Setup an optimizer (e.g., Adam)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_loss, val_loss = train_qa_context_model_boilerplate(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=dev_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=10,
    device=device,
    inputs = ["context_question", "attention_mask_context_question"],
    evaluate_val_dataset=True,
)


In [ ]:
# Plotting the training and validation loss
plot_loss_curves(train_loss, val_loss)

#### Save Model

In [ ]:
model_path = "models/glove_qa_context_model_transformer.pkl"

In [ ]:
save_model(model, model_path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(model_path=model_path)

#### Evaluation

In [ ]:
# evaluate the model on the dev set
criterion = nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
evaluate_qa_context_model_boilerplate(model=model, dataloader=train_dataloader, criterion=criterion, device=device, inputs = ["context_question", "attention_mask_context_question"])

In [ ]:
# evaluate the model on the dev set
criterion = nn.CrossEntropyLoss()
evaluate_qa_context_model_boilerplate(model=model, dataloader=dev_dataloader, criterion=criterion, device=device, inputs = ["context_question", "attention_mask_context_question"])

##### Evaluate predictions on test set

In [ ]:
preds, true_labels = predict_qa_context_model_boilerplate(model=model, dataloader=dev_dataloader, tokenizer=tokenizer, device=device, inputs = ["context_question", "attention_mask_context_question"])

In [ ]:
# Compare the predictions with the actual answers
for i in range(100):
    print(f"Question: {questions[i]}")
    print(f"Context: {contexts[i]}")
    print(f"Predicted Answer: {preds[i]}")
    print(f"True Answer: {true_labels[i]}")
    print("-" * 50)

##### Evaluate predictions on training set

In [ ]:
train_preds, train_true_labels = predict_qa_context_model_boilerplate(model=model, dataloader=train_dataloader, tokenizer=tokenizer, device=device, inputs = ["context_question", "attention_mask_context_question"])

In [ ]:
# Compare the predictions with the actual answers
for i in range(100):
    print(f"Question: {train_questions[i]}")
    print(f"Context: {train_context[i]}")
    print(f"Predicted Answer: {train_preds[i]}")
    print(f"True Answer: {train_true_labels[i]}")
    print("-" * 50)